In [1]:
# coding: utf-8
"""
DM計算用
"""
from library.filer import Filer
from library.mixture_of_unigram_model import MUM
from library.dnp import Evaluation
import glob
import numpy as np

In [4]:
list_filepath = glob.glob('files/rakuten_corpus/rakuten_corpus_master/preprocessedfile/type2/forUM/*.txt')
list_testfile = Filer.readdump('./files/rakuten_corpus/rakuten_corpus_master/testfile/list_sepword_label.dump')
list_testword = [row[0] for row in list_testfile]
list_label = [row[1] for row in list_testfile]

In [ ]:
for path in list_filepath:
    print path
    path_rev = path.replace('files/rakuten_corpus/rakuten_corpus_master/preprocessedfile/type2/forUM/', '')
    Filer.writetxt([path_rev], './files/result/rakuten/experiment3/DM.txt')
    for z in range(3,50):
        K = z
        alpha = 0.01    # 初期ハイパーパラメータalpha
        beta = 0.01     # 初期ハイパーパラメータbeta
        N = 1000        # 最大イテレーション回数
        converge = 0.01 # イテレーション10回ごとに対数尤度を計算し，その差分(converge)が小さければ学習を終了する

        mum = MUM(path)
        mum.set_param(alpha, beta, K, N, converge)
        likelihood = mum.learn()
        Filer.writetxt([str(z) + ' ' + str(likelihood)], './files/result/rakuten/experiment3/DM.txt')
    
        list_predict = []
        for row in list_testword:
            try:
                list_prob = mum.infer(row)
                list_predict.append(np.argmax(list_prob))
            except KeyError:
                list_predict.append(None)

        list_predict_rev = []
        list_label_rev = []
        for predict, label in zip(list_predict, list_label):
            if predict != None:
                list_predict_rev.append(predict)
                list_label_rev.append(label)

        eva = Evaluation()
        dict_result = eva.cal_f_measure(list_predict_rev, list_label_rev)
        Filer.writetxt([' '.join([str(dict_result['purity']),str(dict_result['invpurity']),str(dict_result['fvalue'])])],
                       './files/result/rakuten/experiment3/DM.txt')